# Datenein- und Ausgabe

Die Hauptanwendung von Python (oder jeder anderen Programmiersprache) in eurer späteren Arbeit wir die Verarbeitung und Analyse von Daten sein.

<center>
<img src='slides/img/8_data_life_cycle.png' width='50%' />
</center>

Dazu werdet ihr sowohl selbst erhobene Daten (Simulationen, Messungen) als auch bereits bestehende Daten verwenden. Das Teilen und verfügbar machen Daten ist ein zentraler Bestandteil der wissenschaftlichen Praxis. Deshalb solltet Ihr schon beim Prozessieren der eigenen Daten daran denken, diese vollständig mit [Metadaten](https://de.wikipedia.org/wiki/Metadaten) zu beschreiben.

**Schlechtes Beispiel:**

      0|    1|    2|    3|    4|    5|    6|    7|    8|    9|
    ----------------------------------------------------------
    1.2| 2.34|  4.6| 2.31|567.1|45.24| 4.63|1.855|  4.2|  1.5|


**Gutes Beispiel:**

    Incubator experiment No. 3456
    Date: 13.03.2018
    Contact: Martin Claus <mclaus@geomar.de>
    Bacteria: Escherichia coli
    Substrate: Agar plates
    Cultivation: 10 days following Aaronson et al. (2017)
    
    temperature [°C]   |   0|    1|    2|    3|    4|    5|    6|    7|    8|    9|
    -----------------------------------------------------------------------------
    growth rate [1/day]| 1.2| 2.34|  4.6| 2.31|567.1|45.24| 4.63|1.855|  4.2|  1.5|

---
## Datenformat

Das Datenformat bezeichnet die Art, wie die Daten strukturiert sind und wie sie bei ihrer Verarbeitung zu interpretieren sind. Generell kann man klassifizieren nach:

-   Selbstbeschreibende Datenformate: Metadaten enthalten

-   nicht-selbstbeschreibende Datenformate: Metadaten sind getrennt von Daten

Weiter kann man klassifizieren nach der Art, wie die Datenkodiert sind

-   Binäre Datenformate:
    Daten werden in binärform gespeichert (wie sie im Speicher sind). Das Datenformat (i.e. Anzahl Bits, Datentyp, etc.)
    muss bekannt sein und sollte in den Metadaten enthalten sein
    
-   Textbasierte Datenformate:
    Die Informationen sind als Textzeichen codiert (ASCII, UTF-8 oder ähnliches). Die Codierung und die Struktur der
    Daten muss bekannt sein.
    
Generell ist davon abzuraten, nicht-selbtsbeschreibende binäre Datenformate zu verwenden, da man diese nicht ohne weiteres weiter geben kann.

Im Folgenden werden wir zwei weit verbreitete Datenformate betrachten:

-   CSV (Comma Separated Values): Textbasiertes Datenformat für tabulare Daten
-   NetCDF: selbstbeschreibendes binäres Datenformat für Multidimensionale Tensoren

---

In [ ]:
import numpy as np
import netCDF4 as nc

import matplotlib.pyplot as plt

## CSV

Als Beispiel verwenden wir den [Hurrel North Atlantic Oscillation Index (station based)](https://climatedataguide.ucar.edu/climate-data/hurrell-north-atlantic-oscillation-nao-index-station-based). Der Inhalt der Textdatei hat folgende Struktur: 

In [ ]:
!head -n 5 data/nao_station_monthly.txt
!echo ...
!tail -n 5 data/nao_station_monthly.txt

Die Struktur kann beschrieben werden durch:

-  2 Kopfzeilen
-  13 Spalten (Jahr, Jan., Feb., ...)
-  fehlende Werte sind durch '-999.' markiert

Um die Daten in den Speicher zu laden verwenden wir die Funktion [numpy.genfromtxt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html#numpy.genfromtxt). Falls wir keine fehlenden Werte haben, können wir auch alternativ [numpy.loadtxt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html#numpy.loadtxt) verwenden.

**Aufgabe**: Laden die Daten der Datei 'data/nao_station_monthly.txt'. Maskiere dabei die fehlenden Werte. Dafür brauchst du folgende Parameter der Funktion np.genfromtxt:

-   skipt_header
-   missing_values
-   usemask

Speichere die Daten in einen Array. Versuche danach, die Daten so umzustrukturieren, dass du einen Zeitvektor (1D-array) und einen Datenvektor erhälst, der zeitlich fortlaufend ist.

In [ ]:
data = np.genfromtxt(...)

In [ ]:
plt.plot(time, nao)
plt.xlabel('year')
plt.ylabel('nao')
plt.title('station based monthly NAO index');

---
## NETCDF

[NetCDF](https://www.unidata.ucar.edu/software/netcdf/docs/user_guide.html) ist ein Datenformat für arraybasierte multidimensionale binäre Daten, welches selbstbeschreibend ist. NetCDF ist auch eine Sammelung von Bibliotheken und Werkzeugen, um NetCDF Dateien zu öffnen oder mit Ihnen zu Arbeiten.

### Struktur eine NetCDF Datensatzes
Um sich den Inhalt oder die Struktur eine Datensatzes anzuschauen, verwendet man den Befehl [ncdump](https://www.unidata.ucar.edu/software/netcdf/docs/netcdf_utilities_guide.html#ncdump_guide) in der Linux Kommandozeile. 

In [ ]:
!ncdump -h data/CRUTEM.4.6.0.0.anomalies.nc

Von diesem Output kann man folgende Datenstruktur erkennen:

Ein Datensatz besteht aus:

-   Dimensionen
-   Variablen
-   globalen Attributen

-  Dimensionen haben einen Namen und eine Länge. Um Dimensionen mit Koordinaten zu versehen, existieren Variablen mit dem gleichen Namen.
-  Variablen haben
   -   einen Datentyp
   -   einen Namen
   -   einen shape, definiert durch ein Tuple von Dimensionen
   -   Attribute
-  Attribute haben
   -   einen Namen
   -   einen Datentyp
   -   einen Wert

Beachte die Länge der Dimension `time`. Diese ist `UNLIMITED`. Das bedeuted, dass die Länge der variabel ist und man jederzeit zusätzliche Zeitscheiben anhängen kann. Dies ist praktisch beim Schreiben von Datensätzen. Beim lesen, insbesondere von großen Datensätzen, ist dies mit Geschwindigkeitseinbußen verbunden.

Standardnamen und -werte für Attribute und Variablen sind für klimabezogene Datensätze in den [Climate and Forecast Metadata Conventions (CF conventions)](http://cfconventions.org/) festegelegt.

Nun öffnen wir diesen Datensatz mit Hilfe des [netCDF4](https://unidata.github.io/netcdf4-python/netCDF4/index.html) Pakets. Dies tun wir, indem wir ein Objekt der Klasse [netCDF4.Dataset](https://unidata.github.io/netcdf4-python/netCDF4/index.html#netCDF4.Dataset) erstellen. Wir können uns die Struktur des Datensatzes anschauen, indem wir das Datensatz Objekt printen.

In [ ]:
ds = nc.Dataset('data/CRUTEM.4.6.0.0.anomalies.nc')
print(ds)

Alle Daten und Metadaten des Datensatzes sind als Attribute des Datensatzobjektes verfügbar. Die wichtigsten Attribute sind

-   dimensions:

    Dictionary, welches die Dimensionsnamen auf Ojekte der Klasse [Dimension](https://unidata.github.io/netcdf4-python/netCDF4/index.html#netCDF4.Dimension) abbildet.
    
-   variables:

    Dictionary, welches die Variablenname auf Objekte der Klasse [Variable](https://unidata.github.io/netcdf4-python/netCDF4/index.html#netCDF4.Variable) abbildet. Diese werden verwendet, um Daten zu lesen oder zu schreiben und um
    auf die Attribute der Variable zuzugreifen.
    
Die globalen Attribute sind als Attribute des Datensatzobjekts verfügbar (z.B. `ds.reference`)

**Aufgabe:**
-   Untersuche den Inhalt der Attribute `dimensions` und `variables`. Wie Lang sind die Dimensionen `longitude` und `time`? Welche Einheit und Datentyp hat die Variable `temperature_anomaly` und auf welchen Dimensionen ist sie definiert? 

### Lesen von Daten

Um Daten aus einer Variable in einem netCDF Datensatz in den Speicher zu lesen, wird lediglich das Variablenobjekt indiziert.

**Aufgabe:**
-   Lese den letzten Zeitschritt der Variable `temperature_anomaly` in den Speicher

In [ ]:
t_anom = ...

plt.pcolormesh(
    ds.variables['longitude'][:],
    ds.variables['latitude'][:],
    t_anom
);

**Aufgabe**: Berechne die global gemittelte Temperatur Anomalie. Da die Daten in spärischen Koordinate vorliegen, müssen die einzelnen Datenpunkte mit dem Cosinus der Breite im Bogenmaß gewichtet werden:

$$ t_{global} = \frac{1}{N M}\sum_{j=1}^M\sum_{i=1}^{N} t_{anom}(j, i) \cos(\theta(j))$$

Versuche dabei, die Broadcasting Funktionalität und die Mittelwertfunktion von Numpy zu verwenden!

In [ ]:
lat = ds.variables['latitude'][:]
t_anom = ds.variables['temperature_anomaly'][:]

t_global = ...

In [ ]:
plt.plot(nc.num2date(ds.variables['time'][:], ds.variables['time'].units), t_G);

### Schreiben von netCDF Datensätzen

So leicht NetCDF Datensätze gelesen werden können, so umständlich sind sie zu schreiben. Folgende Schritte sind notwendig, um einen Datensatz zu schreiben:

1.   Datensatzobjekt erstellen und globale Attribute setzen:

```python
ds = nc.Dataset('test.nc', mode='w')
ds.contact = 'Martin Claus <mclaus@geomar.de>'
ds.description = 'Zonally averaged surface temperature from NCEP Reanalysis'
ds.data_source = 'https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.html'
ds.creation_date = '16-06-2019'
```

2. Dimensionen erstellen:

```python
ds.createDimension('latitude', 20)
ds.createDimension('time', None)   # unlimited dimension
```

3. Variablen erstellen:

```python
lon = ds.createVariable('latitude', 'f8', dimensions=('latitude',))
time = ds.createVariable('time', 'f8', dimensions=('time',))
t = ds.createVariable('temperature', 'f8', dimensions=('time', 'longitude')
```

4. Variablendaten schreiben und Attribute anlegen

```python
lon[:] = np.linspace(-80., 80., 20.)
lon.units = "degrees_north"
lon.long_name = "Latitude"

time.units = 'days since 2012-01-01 00:00'
time.calendar = 'gregorian'
time.long_name = 'Time'

t.units = 'degC'
t.long_name = 'zonally averaged temperature'

# write timestamp and data
time[0] = 0.
t[0, :] = ...
```

5. Datensatz schließen (erst jetzt wird auf die Festplatte geschrieben)

```python
ds.close()
```

**Aufgabe:** Schreibe die oben ausgerechnete Temperaturzeitreihe inkl. aller relevanten Metadaten in einen NetCDF Datensatz. Verwende dabei möglichst viele Informationen aus dem Quelldatensatz.

Hinweis: Um einen existierenden Datensatz zu überschreiben, muss man das Argument `clobber=True` beim erstellen des Datensatzes verwenden.

---
## Andere Datenformate


Format   |     Bibliothek
---------|---------------
JSON     |   [json](https://docs.python.org/3/library/json.html)
XML      |   [xml.etree.ElementTree](https://docs.python.org/3/library/xml.etree.elementtree.html)
xls      |   in csv umwandeln oder [pandas](https://pandas.pydata.org/)
mat      |   [scipy.io.loadmat](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html#scipy.io.loadmat)
shp      |   [pyshp](https://pypi.org/project/pyshp/)